In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import string
import re
import joblib
import os

In [ ]:
reviews = pd.read_csv("reviewsClean.csv")

In [ ]:
#All lower case
#Remove numbers
#Remove punctuation
translator = str.maketrans('', '', string.punctuation)
#Remove whitespace

# for i in range(len(reviews)):
#     reviews["reviews"][i] = reviews["reviews"][i].lower()
#     reviews["reviews"][i] = re.sub(r'\d+', '', reviews["reviews"][i])
#     reviews["reviews"][i] = reviews["reviews"][i].translate(translator)
#     reviews["reviews"][i] = " ".join(reviews["reviews"][i].split())
# #print(reviews)
# reviews.to_csv("reviewsClean.csv", index=False)


In [ ]:
print(len(reviews["reviews"][0].split()))
print(round(sum([len(i.split())for i in reviews["reviews"]])/len(reviews["reviews"])))

max_vocab_length = 10000
max_length = 10469

In [ ]:
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length,
                                    pad_to_max_tokens=True)

text_vectorizer.adapt(reviews["reviews"])

In [ ]:
#random_sentence = random.choice(reviews["reviews"])
#print(random_sentence)
#print(text_vectorizer(random_sentence))

#words_in_vocab = text_vectorizer.get_vocabulary()
#top_5_words = words_in_vocab[:5]
#bottom_5_words = words_in_vocab[-5:]
#print(top_5_words,bottom_5_words)

In [ ]:
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=64,
                             input_length=max_length)

#print(embedding(text_vectorizer(random_sentence)))

In [ ]:
pooling_layer = layers.GlobalAveragePooling1D()

In [ ]:
x = text_vectorizer(reviews["reviews"])
x = embedding(x)
x = pooling_layer(x)

In [ ]:
nn = NearestNeighbors(n_neighbors=10)
nn.fit(x)

In [ ]:
# Save model
directory = "saved_model"

if not os.path.exists(directory):
    os.makedirs(directory)

# Define the file path
file_path = os.path.join(directory, "nearest_neighbors.joblib")
joblib.dump(nn, file_path)

In [ ]:
# Load model
if  os.path.exists(directory):
    nn = joblib.load(file_path)

In [ ]:
text = embedding(text_vectorizer("family"))
neighbours = nn.kneighbors(text, return_distance=False)

In [ ]:
for index in neighbours[0]:
    movie_id = reviews.iloc[index]["imdb_id"]
    print(f"Movie ID: {movie_id}")